In [1]:
import os
import datetime
from pprint import pprint
from dotenv import load_dotenv

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation
from prism_agent_open_api_specification_client.types import Response

def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

# Create a connection

The process of establishing a connection between two peers, referred to as the Inviter and Invitee, begins with the Inviter creating an out-of-band (oob) invitation. This invitation contains all the necessary information for the Invitee to connect with the Inviter. Once the invitation is created, it must be handed to the Invitee in some way, such as through email, messaging or QR code. Once the Invitee receives the invitation, they must accept it in order to proceed with the connection. Finally, with the invitation accepted, the connection is established, allowing the two peers to communicate and share data.

## Client instances

We will create two separate clients, one for the inviter and one for the invitee, in order to establish a connection between the two. The roles of the inviter and invitee may overlap with the traditional holder, issuer, and verifier relationships. In some cases, the inviter may also have the role of an issuer or verifier, while the invitee may have the role of a holder. This flexibility allows for different scenarios and use cases to be supported within the same flow

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [2]:
load_dotenv("../BetaProgram/variables.env")
inviterApiKey = os.getenv('ISSUER_APIKEY')
inviterUrl = os.getenv('ISSUER_URL')

inviteeApiKey = os.getenv('HOLDER_APIKEY')
inviteeUrl = os.getenv('HOLDER_URL')

inviter_client = Client(base_url=inviterUrl, headers={"apiKey": inviterApiKey})
invitee_client = Client(base_url=inviteeUrl, headers={"apiKey": inviteeApiKey})

### Inviter Creates the invitation

An invitation is created when a connection is created. The only parameter required is a label to identify the connection with a human-readable format. 

In [3]:
conn_request = CreateConnectionRequest()
conn_request.label = f'Connect {datetime.date.today().strftime("%Y-%m-%d %H:%M:%S")}'
inviter_connection: Response[Connection] =  create_connection.sync(client=inviter_client,json_body=conn_request)

invitation = get_invitation_str(inviter_connection)

Details of the connection on the inviter side are presented below. The connection contains the invitation in the invitation_url field.

In [4]:
print(f"Connection Object:\n") 
pprint(f"{inviter_connection}")
print(f"\nConnection Id: {inviter_connection.connection_id}")
print(f"State: {inviter_connection.state}")
print(f"Label: {inviter_connection.label}")
print(f"OOB Invitation: {get_invitation_str(inviter_connection)}")

Connection Object:

("Connection(self_='Connection', "
 "kind='/connections/85181aba-5443-4106-abb2-671498e21e9c', "
 "connection_id='85181aba-5443-4106-abb2-671498e21e9c', "
 "state=<ConnectionAllOfState.INVITATION_GENERATED: 'InvitationGenerated'>, "
 'created_at=datetime.datetime(2023, 1, 27, 6, 24, 7, 760004, tzinfo=tzutc()), '
 "invitation=ConnectionInvitation(id='85181aba-5443-4106-abb2-671498e21e9c', "
 "type='https://didcomm.org/out-of-band/2.0/invitation', "
 "from_='did:peer:2.Ez6LSpv9T7oZ5Hzuj1jq6Y1qEHDzoZtnQrZMpfgFg5GbCQgdb.Vz6MkmLAEAQdLqD3nSgVpow4jS12L9ocCqUJuVTUfpo7W8VNT.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9iZzY1ai5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', "
 "invitation_url='https://domain.com/path?_oob=eyJpZCI6Ijg1MTgxYWJhLTU0NDMtNDEwNi1hYmIyLTY3MTQ5OGUyMWU5YyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNwdjlUN29aNUh6dWoxanE2WTFxRUhEem9adG5RclpNcGZnRmc1R2JDUWdkYi5WejZNa21

### Invitee accepts the invitation 

When the inviter creates the invitation, there is no connection record on the invitee side, which is why the invitation is shared out of band. Here we conveniently use a variable to pass the initation to the invitee

In [5]:
accept_conn_request = AcceptConnectionInvitationRequest(invitation)
invitee_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=invitee_client,json_body=accept_conn_request)


Details of the connection on the invitee side are presented below.

In [6]:
print(f"Connection Object:\n") 
pprint(f"{invitee_connection}")
print(f"\nConnection Id: {invitee_connection.connection_id}")
print(f"State: {invitee_connection.state}")
print(f"Label: {invitee_connection.label}")
print(f"OOB Invitation: {get_invitation_str(invitee_connection)}")

Connection Object:

("Connection(self_='Connection', "
 "kind='/connections/a1135a04-f195-4df5-bddf-8f69996b06fa', "
 "connection_id='a1135a04-f195-4df5-bddf-8f69996b06fa', "
 'state=<ConnectionAllOfState.CONNECTION_REQUEST_PENDING: '
 "'ConnectionRequestPending'>, created_at=datetime.datetime(2023, 1, 27, 6, "
 '24, 56, tzinfo=tzutc()), '
 "invitation=ConnectionInvitation(id='85181aba-5443-4106-abb2-671498e21e9c', "
 "type='https://didcomm.org/out-of-band/2.0/invitation', "
 "from_='did:peer:2.Ez6LSpv9T7oZ5Hzuj1jq6Y1qEHDzoZtnQrZMpfgFg5GbCQgdb.Vz6MkmLAEAQdLqD3nSgVpow4jS12L9ocCqUJuVTUfpo7W8VNT.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9iZzY1ai5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', "
 "invitation_url='https://domain.com/path?_oob=eyJpZCI6Ijg1MTgxYWJhLTU0NDMtNDEwNi1hYmIyLTY3MTQ5OGUyMWU5YyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNwdjlUN29aNUh6dWoxanE2WTFxRUhEem9adG5RclpNcGZnRmc1R2JDUWdkYi5

### Check Inviter connection

The states of the connection on the inviter side are as described below:

InvitationGenerated --> ConnectionRequestReceived --> ConnectionResponsePending --> **ConnectionResponseSent**

The PRISM Agent does the process to establish the connection automatically, so it may not be possible to track all the protocol steps.

In [7]:
inviter_connection: Response[Connection] =  get_connection.sync(client=inviter_client,connection_id=inviter_connection.connection_id)

In [8]:
print(f"Connection Object:\n") 
pprint(f"{inviter_connection}")
print(f"\nConnection Id: {inviter_connection.connection_id}")
print(f"State: {inviter_connection.state}")
print(f"Label: {inviter_connection.label}")
print(f"OOB Invitation: {get_invitation_str(inviter_connection)}")

Connection Object:

("Connection(self_='Connection', "
 "kind='/connections/85181aba-5443-4106-abb2-671498e21e9c', "
 "connection_id='85181aba-5443-4106-abb2-671498e21e9c', "
 'state=<ConnectionAllOfState.CONNECTION_RESPONSE_SENT: '
 "'ConnectionResponseSent'>, created_at=datetime.datetime(2023, 1, 27, 6, 24, "
 '7, tzinfo=tzutc()), '
 "invitation=ConnectionInvitation(id='85181aba-5443-4106-abb2-671498e21e9c', "
 "type='https://didcomm.org/out-of-band/2.0/invitation', "
 "from_='did:peer:2.Ez6LSpv9T7oZ5Hzuj1jq6Y1qEHDzoZtnQrZMpfgFg5GbCQgdb.Vz6MkmLAEAQdLqD3nSgVpow4jS12L9ocCqUJuVTUfpo7W8VNT.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9iZzY1ai5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', "
 "invitation_url='https://domain.com/path?_oob=eyJpZCI6Ijg1MTgxYWJhLTU0NDMtNDEwNi1hYmIyLTY3MTQ5OGUyMWU5YyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNwdjlUN29aNUh6dWoxanE2WTFxRUhEem9adG5RclpNcGZnRmc1R2JDUWdkYi5WejZN

### Check Invitee connection

The states of the connection on the inviter side are as described below:

InvitationReceived  --> ConnectionRequestPending  --> ConnectionRequestSent     --> **ConnectionResponseReceived**

The PRISM Agent does the process to establish the connection automatically, so it may not be possible to track all the protocol steps.

In [9]:
invitee_connection: Response[Connection] =  get_connection.sync(client=invitee_client,connection_id=invitee_connection.connection_id)

In [10]:
print(f"Connection Object:\n") 
pprint(f"{invitee_connection}")
print(f"\nConnection Id: {invitee_connection.connection_id}")
print(f"State: {invitee_connection.state}")
print(f"Label: {invitee_connection.label}")
print(f"OOB Invitation: {get_invitation_str(invitee_connection)}")

Connection Object:

("Connection(self_='Connection', "
 "kind='/connections/a1135a04-f195-4df5-bddf-8f69996b06fa', "
 "connection_id='a1135a04-f195-4df5-bddf-8f69996b06fa', "
 'state=<ConnectionAllOfState.CONNECTION_RESPONSE_RECEIVED: '
 "'ConnectionResponseReceived'>, created_at=datetime.datetime(2023, 1, 27, 6, "
 '24, 56, tzinfo=tzutc()), '
 "invitation=ConnectionInvitation(id='85181aba-5443-4106-abb2-671498e21e9c', "
 "type='https://didcomm.org/out-of-band/2.0/invitation', "
 "from_='did:peer:2.Ez6LSpv9T7oZ5Hzuj1jq6Y1qEHDzoZtnQrZMpfgFg5GbCQgdb.Vz6MkmLAEAQdLqD3nSgVpow4jS12L9ocCqUJuVTUfpo7W8VNT.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9iZzY1ai5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', "
 "invitation_url='https://domain.com/path?_oob=eyJpZCI6Ijg1MTgxYWJhLTU0NDMtNDEwNi1hYmIyLTY3MTQ5OGUyMWU5YyIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNwdjlUN29aNUh6dWoxanE2WTFxRUhEem9adG5RclpNcGZnRmc1R2JDUWd

## List all connections

The following code retrieves the lists of connections.

In [11]:
inviter_connections: Response[ConnectionCollection] = get_connections.sync(client=inviter_client)
invitee_connections: Response[ConnectionCollection] = get_connections.sync(client=invitee_client)
print("Inviter connections")
print("-------------------\n")
pprint("{}".format(inviter_connections, invitee_connections))
print("\nInvitee connections")
print("-------------------\n")
pprint("{}".format(invitee_connections, invitee_connections))

Inviter connections
-------------------

("ConnectionCollection(self_='/collections', kind='Collection', "
 "contents=[Connection(self_='Connection', "
 "kind='/connections/db1b8dc2-fac3-4e48-9fb1-696b599022cb', "
 "connection_id='db1b8dc2-fac3-4e48-9fb1-696b599022cb', "
 "state=<ConnectionAllOfState.INVITATION_GENERATED: 'InvitationGenerated'>, "
 'created_at=datetime.datetime(2023, 1, 24, 20, 31, 59, tzinfo=tzutc()), '
 "invitation=ConnectionInvitation(id='db1b8dc2-fac3-4e48-9fb1-696b599022cb', "
 "type='https://didcomm.org/out-of-band/2.0/invitation', "
 "from_='did:peer:2.Ez6LSkSZoWh97GSLE4Gu1A7MjZEEQXiBuAmEXRxDYW4Z5bVD6.Vz6MkmavqzCrTy9ztrgXawZwSn5hKtN89p8fLkYGH36jCYsPV.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9iZzY1ai5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19', "
 "invitation_url='https://domain.com/path?_oob=eyJpZCI6ImRiMWI4ZGMyLWZhYzMtNGU0OC05ZmIxLTY5NmI1OTkwMjJjYiIsInR5cGUiOiJodHRwczovL2RpZGNvbW0ub3JnL291dC1vZi1iYW5kLzIuMC9pbnZpdGF0aW9uIiwiZnJvbSI6